In [86]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly
import re

# Postulantes/Educacion

In [87]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ]
})
valores_educacion.to_csv('../data/adri_data/valor_educacion.csv', sep=',', encoding='utf-8', index=False)
# valores_educacion

In [88]:
# Carga de archivos
post_educacion_1  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion.csv')
#post_educacion.drop('Unnamed: 0', axis=1, inplace=True)

# Carga de archivos
post_educacion_2  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_2.csv')

# Carga de archivos
post_educacion_3  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_3.csv')

post_educacion = pd.concat([post_educacion_1, post_educacion_2, post_educacion_3], axis=0)
# post_educacion.duplicated().sum() # 231551
post_educacion.count()              # 1103193


idpostulante    1103193
nombre          1103193
estado          1103193
dtype: int64

In [89]:
# post_educacion.drop_duplicates(inplace = True)

# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_educacion = pd.merge(post_educacion, valores_educacion, how='inner', \
                         left_on=['nombre','estado'], right_on = ['nombre','estado'])

# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
max_post_educacion.count() # 447909

idpostulante       447909
valor_educacion    447909
dtype: int64

In [90]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner', \
                         left_on=['idpostulante','valor_educacion'], \
                         right_on = ['idpostulante','valor_educacion'])

post_educacion.drop(['nombre', 'estado'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

post_educacion.to_csv('../data/adri_data/postulantes_educacion.csv', sep=',', encoding='utf-8', index=False)
post_educacion.count()

idpostulante       447909
valor_educacion    447909
dtype: int64

# Postulantes/Genero/Edad

In [91]:
post_gen_edad_1 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad.csv')
post_gen_edad_2 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_2.csv')
post_gen_edad_3 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_3.csv')

post_gen_edad = pd.concat([post_gen_edad_1, post_gen_edad_2, post_gen_edad_3], axis=0)
post_gen_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [92]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')
post_gen_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 200887
Data columns (total 3 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null category
dtypes: category(1), object(2)
memory usage: 18.6+ MB


In [93]:
post_gen_edad.drop_duplicates(inplace=True)
post_gen_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [94]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

post_gen_edad.count() 

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [95]:
# Modifico el tipo de dato de la fecha.
post_gen_edad.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

post_gen_edad.to_csv('../data/adri_data/postulantes_genero_edad.csv', sep=',', encoding='utf-8', index=False)

In [96]:
post_gen_edad = pd.read_csv('../data/adri_data/postulantes_genero_edad.csv')
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 
post_gen_edad.count() # 504407 
# post_gen_edad.count() # 505382 

idpostulante    504407
sexo            504407
edad            477883
dtype: int64

In [97]:
post_gen_edad.fillna(-1, inplace=True)
post_gen_edad['edad'] = post_gen_edad['edad'].astype(int)
post_gen_edad

,idpostulante,sexo,edad
0,6MM,2,33
1,Nzz,-1,-1
2,ZX1,-1,-1
3,Nq5,-1,-1
4,ebE,2,65
5,N1x,-1,-1
6,52aw,-1,-1
7,NAjM,1,56
8,eRk9,-1,-1
9,a6OE,-1,-1


# Merge de Postulantes

In [101]:
postulantes = pd.merge(post_educacion, post_gen_edad, how='outer', on='idpostulante')
postulantes.shape

(504407, 4)

In [ ]:
# inner: 447909
# outer: 504407

In [105]:
postulantes.fillna(-1, inplace=True)

In [106]:
postulantes.to_csv('../data/adri_data/postulantes.csv', sep=',', encoding='utf-8', index=False)

In [14]:
# postulantes.dropna(axis=0, inplace=True)
# postulantes[(postulantes['sexo'] == -1) | (postulantes['sexo'].isnull())]

In [15]:
# postulantes[postulantes['edad'] == -1]

In [16]:
# postulantes[postulantes.duplicated('idpostulante')]

# Vistas

In [103]:
vistas_1 = pd.read_csv('../data/tp2_data/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_2.csv')
vistas_3 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_3.csv')

vistas = pd.concat([vistas_1, vistas_2, vistas_3], axis=0)
vistas.drop_duplicates(inplace=True)
vistas.rename({'idAviso' : 'idaviso'}, inplace =True, axis=1)

In [104]:
vistas['timestamp'] = vistas['timestamp'].apply(pd.to_datetime)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)

vistas['timestamp'] = (now - vistas['timestamp']).apply(lambda x: x.total_seconds())

In [ ]:
vistas.to_csv('../data/adri_data/vistas.csv', sep=',', encoding='utf-8', index=False)

# Postulaciones

In [21]:
postulaciones_1 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones_2.csv')
postulaciones = pd.concat([postulaciones_1, postulaciones_2], axis=0)

postulaciones.drop_duplicates(inplace=True)

In [22]:
postulaciones.rename({'fechapostulacion': 'timestamp'}, axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,timestamp
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [23]:
postulaciones['timestamp'] = postulaciones['timestamp'].apply(pd.to_datetime)

In [24]:
# now = pd.Timestamp(dt.datetime.now())
# pd.to_datetime(now)

postulaciones['timestamp'] = (now - postulaciones['timestamp']).apply(lambda x: x.total_seconds())

In [25]:
postulaciones.to_csv('../data/adri_data/postulaciones.csv', sep=',', encoding='utf-8', index=False)

postulaciones.head()

,idaviso,idpostulante,timestamp
0,1112248724,NjlD,1.250329e+07
1,1112286523,ZaO5,1.204439e+07
2,1112272060,ZaO5,1.204364e+07
3,1112288401,ZaO5,1.189502e+07
4,1112300563,ZaO5,1.153150e+07


# Avisos Online

In [110]:
avisos_online_1 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online.csv')
avisos_online_2 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online_2.csv')
avisos_online = pd.concat([avisos_online_1, avisos_online_2], axis=0)

avisos_online.drop_duplicates(inplace=True)
avisos_online.to_csv('../data/adri_data/avisos_online.csv', sep=',', encoding='utf-8', index=False)

avisos_online.head()

,idaviso
0,1111718795
1,1110497155
2,1111511125
3,1111930643
4,1112172032


In [27]:
# avisos_online['idaviso'].unique().shape

In [28]:
# top100k = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
#top100k['idaviso'].unique().shape

# Detalle Avisos

In [29]:
avisos_detalles_1 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_2.csv')
avisos_detalles_3 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_3.csv')
avisos_detalles_4 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_4.csv')
avisos_detalles = pd.concat([avisos_detalles_1, avisos_detalles_2, avisos_detalles_3, avisos_detalles_4], axis=0)

In [30]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop_duplicates('idaviso', inplace=True)

In [31]:
avisos_detalles['titulo'] = avisos_detalles['titulo'].apply(lambda x: x.strip())
avisos_detalles.count()

idaviso                 25288
titulo                  25288
descripcion             25288
nombre_zona             25288
ciudad                    160
mapacalle                2069
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [32]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [33]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(14, 10)

In [34]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica','nombre_area'] = 'Bioquimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(0, 10)

In [35]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(9, 10)

In [36]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química','nombre_area'] = 'Quimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(0, 10)

In [37]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(3, 10)

In [38]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social','nombre_area'] = 'Trabajo Social'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(0, 10)

In [39]:
# areas = pd.DataFrame(avisos_detalles['nombre_area'].unique())
# areas.to_csv('../data/adri_data/areas_intermedio.csv', encoding='utf-8', index=False)
# areas

In [40]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [41]:
# avisos_detalles.duplicated().sum()

In [42]:
# avd = avisos_detalles[avisos_detalles.duplicated('idaviso')]['idaviso'].reset_index()
# avd = pd.merge(avd, avisos_detalles)
# avd.to_csv('../data/tp2_data/duplex.csv')

In [43]:
# zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique())
# zona

In [44]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalle_0.csv', sep=',', encoding='utf-8', index=False)

# Despues de la Unificacion

In [195]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalle_0.csv', encoding='utf-8')
avisos_detalles.shape

(25288, 10)

In [196]:
avisos_online['esta_online'] = 1

In [197]:
avisos_detalles = pd.merge(avisos_detalles, avisos_online, how='left', on='idaviso')

In [198]:
avisos_detalles['esta_online'] = avisos_detalles['esta_online'].apply(lambda x: 1 if x == 1 else 0)
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0


In [199]:
def cleanText(t):
    t = t.lower()\
        .replace('"', ' ') \
        .replace('*', ' ') \
        .replace('(', ' ') \
        .replace(')', ' ') \
        .replace('!', ' ') \
        .replace('¡', ' ') \
        .replace('¿', ' ') \
        .replace('?', ' ') \
        .replace('-', ' ') \
        .replace('/', ' ') \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \
        .replace('ë', 'e') \
        .replace('ü', 'u') \
        .replace(',', ' ') \
        .replace('.', ' ') \
        .replace('-', ' ') \
        .replace('=', ' ') \
        .replace('&nbsp;', ' ') \
        .replace('&amp;', ' ') \
        .replace('&quot;', ' ') \
        .replace('&iquest;', ' ') \
        .replace('&lt;', ' ') 
    t = re.sub(r"(</?[\w\d\s]+>)", " ", t)
    t = t\
        .replace('<', ' ') \
        .replace('>', ' ') \
        .replace('\r', ' ') \
        .replace('\n', ' ')
        
    return t.strip()

def cleanCollection(s):
    s['palabra'] = s['palabra'].str.strip()
    s = s[s['palabra'] != '']
    s = s[s['palabra'] != 'de']
    s = s[s['palabra'] != 'para']
    s = s[s['palabra'] != 'y']
    s = s[s['palabra'] != 'a']
    s = s[s['palabra'] != 'la']
    s = s[s['palabra'] != 'en']
    s = s[s['palabra'] != 'el']
    s = s[s['palabra'] != 'con']
    s = s[s['palabra'] != 'zona']
    s = s[s['palabra'] != 'o']
    s = s[s['palabra'] != 'los']
    s = s[s['palabra'] != 'se']
    s = s[s['palabra'] != 'del']
    s = s[s['palabra'] != 'las']
    s = s[s['palabra'] != 'un']
    s = s[s['palabra'] != 'al']
    s = s[s['palabra'] != 'una']
    s = s[s['palabra'] != 'que']
    s = s[s['palabra'] != 'busqueda']
    s = s[s['palabra'] != 'por']
    s = s[s['palabra'] != 'años']
    s = s[s['palabra'] != 'es']
    s = s[s['palabra'] != '·']
    s = s[s['palabra'] != 'e']
    s = s[s['palabra'] != 'sus']
    s = s[s['palabra'] != 'no']
    s = s[s['palabra'] != 'and']
    s = s[s['palabra'] != 'su']
    s = s[s['palabra'] != 'como']
    s = s[s['palabra'] != 'area']
    s = s[s['palabra'] != 'nos']
    s = s[s['palabra'] != 'lunes']
    s = s[s['palabra'] != 'nuestro']
    s = s[s['palabra'] != 'poder']
    s = s[s['palabra'] != 'mismo']
    return s

In [200]:
avisos_detalles['titulo'] = avisos_detalles['titulo'].apply(lambda x:cleanText(x))
avisos_detalles['descripcion'] = avisos_detalles['descripcion'].apply(lambda x:cleanText(x))

In [201]:
avisos_detalles['descripcion']

0        buscamos un  responsable de producto  para inc...
1        para nuestro cliente  concesionario chevrolet ...
2        importante estudio de auditoria y consultoria ...
3        chevron is a multinational company in the ener...
4        importante empresa de energias renovables  se ...
5        importante industria de impresion ubicada en l...
6        importante empresa ubicada en la zona de don t...
7        importante colegio bilingue de olivos  busca i...
8        importante logistica de la zona de palomar inc...
9        para importante empresa de call center  zona c...
10       buscamos un ingeniero mecanico o electromecani...
11       importante empresa de servicios para obras de ...
12       logan travel  operador mayorista de turismo  q...
13       estamos buscando dos  2  vendedoras para distr...
14       importante hipermercado se encuentra en la bus...
15       del castillo   rossi advisors human capital   ...
16       p  p    la empresa numero uno a nivel mundial .

In [202]:
avisos_detalles['titulo']

0        responsable de marketing producto div  constru...
1                asesor de servicios   postventa chevrolet
2                       analistas de impuestos sr ssr y jr
3                                        software engineer
4                               business developer manager
5                       tecnico quimico mecanico  arroyito
6        administrativo de creditos y finanzas junior  ...
7                    docente de ingles turno tarde  olivos
8                       m   analista de control de gestion
9                  representantes de cobranzas telefonicas
10                    ingeniero mecanico o electromecanico
11       detectorista y operador para tunelera inteligente
12                           operador mayorista   vendedor
13       se busca vendedoras para distribuidora mayoris...
14                                     atencion al cliente
15                 vendedor de planes de ahorro   camiones
16       telemarketer con experiencia en ventas   turno.

### Detalle Avisos (Titulo)

In [203]:
avisos_detalles_titulo = pd.DataFrame(avisos_detalles['titulo'].unique(), columns=['titulo'])
avisos_detalles_titulo = avisos_detalles_titulo.sort_values('titulo').reset_index(drop=True)
avisos_detalles_titulo.head()

,titulo
0,
1,03 mar chief software architect usd$ 100k y...
2,03 mar cloud chief software architect usd$1...
3,03 mar group controller $200k year onlin...
4,03 mar inside sales representative usd$ 100...


In [204]:
avisos_detalles_descripcion = pd.DataFrame(avisos_detalles['descripcion'].unique(), columns=['descripcion'])
avisos_detalles_descripcion = avisos_detalles_descripcion.sort_values('descripcion').reset_index(drop=True)
avisos_detalles_descripcion.head()

,descripcion
0,
1,+25 edad experiencia en el rubro experiencia c...
2,+vida es una organizacion especializada en br...
3,079 dru ingi ingeniero industrial empresa d...
4,1 tecnicos quimicos con experiencia secunda...


In [205]:
joined_titulo = ' '.join(avisos_detalles_titulo['titulo'])
joined_descripcion = ' '.join(avisos_detalles_descripcion['descripcion'])

In [206]:
splitted = joined_titulo.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)

splitted['length'] = splitted['palabra'].apply(lambda x: len(x))

splitted = cleanCollection(splitted)

splitted.to_csv('../data/adri_data/palabras_titulo.csv', encoding = 'utf-8', index=False)
splitted = splitted.reset_index(drop=True)
splitted = splitted[splitted['count'] > 500]
splitted

,palabra,count,length
0,analista,2231,8
1,tecnico,1158,7
2,administrativo,924,14
3,vendedor,885,8
4,comercial,815,9
5,ventas,755,6
6,sr,696,2
7,importante,676,10
8,jefe,603,4
9,empresa,583,7


In [207]:
for x in splitted.palabra:
    avisos_detalles['titulo_' + x] = avisos_detalles['titulo'].apply(lambda y: 1 if x in y else 0)    

In [208]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,...,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,titulo_jefe,titulo_empresa,titulo_asistente,titulo_supervisor,titulo_jr,titulo_contable
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,...,0,0,0,0,0,0,0,0,0,0
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,...,0,0,0,0,0,0,0,0,0,0
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,...,0,0,1,0,0,0,0,0,1,0
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,...,0,0,0,0,0,0,0,0,0,0


In [209]:
splitted = joined_descripcion.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)

splitted['length'] = splitted['palabra'].apply(lambda x: len(x))

splitted = cleanCollection(splitted)

splitted.to_csv('../data/adri_data/palabras_descripcion.csv', encoding = 'utf-8', index=False)
splitted = splitted.reset_index(drop=True)
splitted = splitted[splitted['count'] > 8000]
splitted

,palabra,count,length
0,experiencia,27065,11
1,empresa,15209,7
2,trabajo,11175,7
3,excluyente,10507,10
4,manejo,9934,6
5,importante,9874,10
6,equipo,9849,6
7,tareas,9279,6
8,cliente,8950,7
9,clientes,8365,8


In [210]:
for x in splitted.palabra:
    avisos_detalles['desc_' + x] = avisos_detalles['descripcion'].apply(lambda y: 1 if x in y else 0)  

In [211]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,...,desc_empresa,desc_trabajo,desc_excluyente,desc_manejo,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,...,1,1,0,0,0,0,1,1,1,1
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,...,0,1,1,1,0,1,1,1,0,0
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,...,0,0,0,1,1,0,0,0,0,0
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,...,1,0,0,0,1,0,0,1,1,1


# Detalle Avisos (Descripcion)

In [212]:
# avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_titulo, how='inner', on='titulo')
avisos_detalles.drop(['titulo', 'descripcion'], inplace=True, axis=1)
avisos_detalles.count()

idaviso                  25288
nombre_zona              25288
ciudad                     160
mapacalle                 2069
tipo_de_trabajo          25288
nivel_laboral            24953
nombre_area              25288
denominacion_empresa     25281
esta_online              25288
titulo_analista          25288
titulo_tecnico           25288
titulo_administrativo    25288
titulo_vendedor          25288
titulo_comercial         25288
titulo_ventas            25288
titulo_sr                25288
titulo_importante        25288
titulo_jefe              25288
titulo_empresa           25288
titulo_asistente         25288
titulo_supervisor        25288
titulo_jr                25288
titulo_contable          25288
desc_experiencia         25288
desc_empresa             25288
desc_trabajo             25288
desc_excluyente          25288
desc_manejo              25288
desc_importante          25288
desc_equipo              25288
desc_tareas              25288
desc_cliente             25288
desc_cli

In [213]:
avisos_detalles.head()

,idaviso,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_analista,...,desc_empresa,desc_trabajo,desc_excluyente,desc_manejo,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos
0,1111556097,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,...,1,1,0,0,0,0,1,1,1,1
1,1111949392,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,...,0,1,1,1,0,1,1,1,0,0
2,1112145935,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0,1,...,0,0,0,1,1,0,0,0,0,0
3,1112146010,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,...,1,0,0,0,1,0,0,1,1,1


### Detalle Avisos (Nombre Zona)

In [214]:
avisos_detalles_zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique(), columns=['nombre_zona'])
avisos_detalles_zona.index.name = 'idzona'
avisos_detalles_zona.to_csv('../data/adri_data/avisos_detalle_zona.csv', sep=',', encoding='utf-8')
avisos_detalles_zona = pd.read_csv('../data/adri_data/avisos_detalle_zona.csv')
avisos_detalles_zona.head()

,idzona,nombre_zona
0,0,Gran Buenos Aires
1,1,Capital Federal
2,2,GBA Oeste
3,3,Buenos Aires (fuera de GBA)
4,4,La Plata


In [215]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_zona, how='inner', on='nombre_zona')
avisos_detalles.drop('nombre_zona', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,...,desc_trabajo,desc_excluyente,desc_manejo,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona
0,1111556097,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,...,1,0,0,0,0,1,1,1,1,0
1,1111949392,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,...,1,1,1,0,1,1,1,0,0,0
2,1112145935,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0,1,0,...,0,0,1,1,0,0,0,0,0,0
3,1112146010,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,...,0,0,0,1,0,0,1,1,1,0


### Detalle Avisos (Nombre Ciudad)

In [216]:
avisos_detalles_ciudad = pd.DataFrame(avisos_detalles['ciudad'].unique(), columns=['ciudad'])
avisos_detalles_ciudad.index.name = 'idciudad'
avisos_detalles_ciudad.to_csv('../data/adri_data/avisos_detalle_ciudad.csv', sep=',', encoding='utf-8')
avisos_detalles_ciudad = pd.read_csv('../data/adri_data/avisos_detalle_ciudad.csv')
avisos_detalles_ciudad.head()

,idciudad,ciudad
0,0,NaN
1,1,Buenos Aires
2,2,Argentina
3,3,caba
4,4,paternal


In [217]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_ciudad, how='inner', on='ciudad')
avisos_detalles.drop('ciudad', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,...,desc_excluyente,desc_manejo,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad
0,1111556097,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,...,0,0,0,0,1,1,1,1,0,0
1,1111949392,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,...,1,1,0,1,1,1,0,0,0,0
2,1112145935,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
3,1112146010,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,0,...,0,0,1,0,0,1,1,1,0,0


### Detalle Avisos (Mapa Calle)

In [218]:
avisos_detalles_mapacalle = pd.DataFrame(avisos_detalles['mapacalle'].unique(), columns=['mapacalle'])
avisos_detalles_mapacalle.index.name = 'idmapacalle'
avisos_detalles_mapacalle.to_csv('../data/adri_data/avisos_detalle_mapacalle.csv', sep=',', encoding='utf-8')
avisos_detalles_mapacalle = pd.read_csv('../data/adri_data/avisos_detalle_mapacalle.csv')
avisos_detalles_mapacalle.head()

,idmapacalle,mapacalle
0,0,NaN
1,1,Guemes 4747
2,2,Comodoro Rivadavia
3,3,Buenos Aires
4,4,Av Hipolito Yrigoyen 13500


In [219]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_mapacalle, how='inner', on='mapacalle')
avisos_detalles.drop('mapacalle', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,...,desc_manejo,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle
0,1111556097,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,...,0,0,0,1,1,1,1,0,0,0
1,1111949392,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,0,...,1,0,1,1,1,0,0,0,0,0
2,1112145935,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,1112146010,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,0,0,...,0,1,0,0,1,1,1,0,0,0


### Detalle Avisos (Tipo de Trabajo)

In [220]:
avisos_detalles_tipo_de_trabajo = pd.DataFrame(avisos_detalles['tipo_de_trabajo'].unique(), columns=['tipo_de_trabajo'])
avisos_detalles_tipo_de_trabajo.index.name = 'idtipo_de_trabajo'
avisos_detalles_tipo_de_trabajo.to_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv', sep=',', encoding='utf-8')
avisos_detalles_tipo_de_trabajo = pd.read_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv')
avisos_detalles_tipo_de_trabajo.head()

,idtipo_de_trabajo,tipo_de_trabajo
0,0,Full-time
1,1,Part-time
2,2,Temporario
3,3,Por Contrato
4,4,Teletrabajo


In [221]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_tipo_de_trabajo, how='inner', on='tipo_de_trabajo')
avisos_detalles.drop('tipo_de_trabajo', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,...,desc_importante,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo
0,1111556097,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0
1,1111949392,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
2,1112145935,Senior / Semi-Senior,Impuestos,Lighthouse Smart,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1112146010,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,Senior / Semi-Senior,Comercial,RANDSTAD,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0


### Detalle Avisos (Nivel Laboral)

In [222]:
avisos_detalles_nivel_laboral = pd.DataFrame(avisos_detalles['nivel_laboral'].unique(), columns=['nivel_laboral'])
avisos_detalles_nivel_laboral.index.name = 'idnivel_laboral'
avisos_detalles_nivel_laboral.to_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv', sep=',', encoding='utf-8')
avisos_detalles_nivel_laboral = pd.read_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv')
avisos_detalles_nivel_laboral.head()

,idnivel_laboral,nivel_laboral
0,0,Senior / Semi-Senior
1,1,Junior
2,2,Jefe / Supervisor / Responsable
3,3,Otro
4,4,Gerencia / Alta Gerencia / Dirección


In [223]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nivel_laboral, how='inner', on='nivel_laboral')
avisos_detalles.drop('nivel_laboral', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,nombre_area,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,...,desc_equipo,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral
0,1111556097,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0
1,1111949392,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
2,1112145935,Impuestos,Lighthouse Smart,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1112146010,Programación,Chevron Buenos Aires Shared Services Center,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1112211475,Comercial,RANDSTAD,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0


### Detalle Avisos (Nombre Area)

In [224]:
avisos_detalles_nombre_area = pd.DataFrame(avisos_detalles['nombre_area'].unique(), columns=['nombre_area'])
avisos_detalles_nombre_area.sort_values('nombre_area', inplace=True)
avisos_detalles_nombre_area.reset_index(inplace=True)
avisos_detalles_nombre_area.drop('index', axis=1, inplace=True)
avisos_detalles_nombre_area.head()

,nombre_area
0,Abastecimiento
1,Administracion de Seguros
2,Administración
3,Administración de Base de Datos
4,Administración de Personal


In [225]:
avisos_detalles_nombre_area.index.name = 'idnombre_area'
avisos_detalles_nombre_area.to_csv('../data/adri_data/avisos_detalle_nombre_area.csv', sep=',', encoding='utf-8')
avisos_detalles_nombre_area = pd.read_csv('../data/adri_data/avisos_detalle_nombre_area.csv')
avisos_detalles_nombre_area.head()

,idnombre_area,nombre_area
0,0,Abastecimiento
1,1,Administracion de Seguros
2,2,Administración
3,3,Administración de Base de Datos
4,4,Administración de Personal


In [226]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nombre_area, how='inner', on='nombre_area')
avisos_detalles.drop('nombre_area', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,denominacion_empresa,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,...,desc_tareas,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area
0,1111556097,Sinteplast S.A.,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,50
1,1112408768,Consultores de Empresas SRL,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50
2,1112408781,Consultores de Empresas SRL,1,1,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,50
3,1112344682,Kaizen Recursos Humanos,1,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,50
4,1112279443,Re/Max Net,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,50


### Detalle Avisos (Denominacion Empresa)

In [227]:
avisos_detalles_denominacion_empresa = pd.DataFrame(avisos_detalles['denominacion_empresa'].unique(), columns=['denominacion_empresa'])
avisos_detalles_denominacion_empresa.head()

,denominacion_empresa
0,Sinteplast S.A.
1,Consultores de Empresas SRL
2,Kaizen Recursos Humanos
3,Re/Max Net
4,F & C Consultores y Asociados


In [228]:
avisos_detalles_denominacion_empresa.index.name = 'iddenominacion_empresa'
avisos_detalles_denominacion_empresa.to_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv', sep=',', encoding='utf-8')
avisos_detalles_denominacion_empresa = pd.read_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv')
avisos_detalles_denominacion_empresa.head()

,iddenominacion_empresa,denominacion_empresa
0,0,Sinteplast S.A.
1,1,Consultores de Empresas SRL
2,2,Kaizen Recursos Humanos
3,3,Re/Max Net
4,4,F & C Consultores y Asociados


In [229]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_denominacion_empresa, how='inner', on='denominacion_empresa')
avisos_detalles.drop('denominacion_empresa', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,50,0
1,1112448447,1,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,182,0
2,1112394600,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,520,0,0,34,0
3,1111266888,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,142,0
4,1112426827,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,114,0


In [230]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalles.csv', sep=',', encoding='utf-8', index=False)

In [231]:
avisos_detalles.count()

idaviso                   25288
esta_online               25288
titulo_analista           25288
titulo_tecnico            25288
titulo_administrativo     25288
titulo_vendedor           25288
titulo_comercial          25288
titulo_ventas             25288
titulo_sr                 25288
titulo_importante         25288
titulo_jefe               25288
titulo_empresa            25288
titulo_asistente          25288
titulo_supervisor         25288
titulo_jr                 25288
titulo_contable           25288
desc_experiencia          25288
desc_empresa              25288
desc_trabajo              25288
desc_excluyente           25288
desc_manejo               25288
desc_importante           25288
desc_equipo               25288
desc_tareas               25288
desc_cliente              25288
desc_clientes             25288
desc_conocimientos        25288
idzona                    25288
idciudad                  25288
idmapacalle               25288
idtipo_de_trabajo         25288
idnivel_

In [232]:
avisos_detalles[avisos_detalles['idaviso']==739260]

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
17330,739260,0,0,0,0,0,1,0,0,0,...,1,1,0,1,0,0,0,2,29,854
